Tutorial: Comparing cycles of a cyclic voltammogram
====================================

*Internet connection required*: This tutorial reads data from a github repository [here](https://github.com/ixdat/tutorials/tree/43e85d07254e67e950f9c7081ffe1fa7f053cc08/L3_data_structure/exports).

Here we will show how to use `ixdat` and python to analyze two cases where the difference in integrated current between two cycles in cyclic voltammatry is needed. 

Using the CO stripping example, we will show three ways of visualizing the stripping experiment and determining the surface area of a platinum electrode. Each gives the amount of charge associated with oxidation of adsorbed CO, and thus an estimate of the electrochemical surface area of the electrode. From most generalized to most automated, they are:

- Finding the right timespans, getting the data as numpy vectors with `grab()`, and integrating with `trapz()`
- Selecting the data with `CyclicVoltammagram` indexing and `select_sweep()`, and integrating with `integrate()`
- Using `subtract()` to get a `CyclicVoltammagramDiff` object that does the analysis.

Then, you will, on your own, choose whichever method you like to calculate the amount of charge associated with the reduction of an oxide layer, which gives an estimate of the thickness of the oxide layer.

Setup
--------
We'll use `numpy` and `pathlib.Path` as well as `ixdat`'s `CyclicVoltammogram` measurement type.
You can always read your data with the basic Measruement class, using `from ixdat import Measurement` and `Measurement.read(...)` object. Ixdat will then determine the best specific class to use. However, if you know exactly which class you want (here `CyclicVoltammogram`), you can use the `read` method of that class.

In [ ]:
import numpy as np
from pathlib import Path

from ixdat.techniques import CyclicVoltammogram 

Loading raw data
----------------------

Using ixdat's `Measurement.read()` method reads a file from your hard drive to make a `Measurement` object. `read_url()` does the same thing, but from a url. When the origin of the file can't be inferred from the extension (e.g., many softwares can make ".csv" files), you should specify the "reader" to tell ixdat how to parse the file. Here, the file is something that was exported from ixdat itself, so the reader is "ixdat".

In [ ]:
if True:  # Set this to False for offline work (requires you have downloaded the data file.)
    co_strip = CyclicVoltammogram.read_url(
        "https://raw.githubusercontent.com/ixdat/tutorials/43e85d07254e67e950f9c7081ffe1fa7f053cc08/L3_data_structure/exports/co_strip.csv",
        reader="ixdat"
    )
else:
    co_strip = CyclicVoltammogram.read(
        "./data/co_strip.csv",
        reader="ixdat"
    )

(Don't worry about the line skipping above, that is because the file was exported with an earlier version of ixdat.)

It's always an option to directly plot anything you read into ixdat. For a `CyclicVoltammogram` object, the default plotting method is current ("J") vs potential ("V"):

In [ ]:
co_strip.plot()

Selecting and calibrating
----------------------------------

`ixdat` supports several types of simple data treating (calibrating, filtering, and background subtraction). `ECMeasurement`s and derived classes like `CyclicVoltammogram` support reference electrode calibration (give `RE_vs_RHE` in \[V\]) current normalization (give `A_el` in \[cm^2\]), ohmic drop correction (give `R_Ohm` in \[Ohm\]). Note that the potential was already calibrated, so we don't calibrate it again here.

In [ ]:
co_strip.calibrate(
    A_el=0.196, 
    R_Ohm=100
)
help(co_strip.calibrate)

Numpy arrays are available with the `grab` method of any ixdat `Measurement`. This method returns two arrays - a time vector and a vector for the value you ask for.

For an `ECMeasurement`, you can always grab `potential` and `current`. These will be the most up-to-date (calibrated) values avialable.

In [ ]:
co_strip.grab("potential")   # Returns the time in [s] and the the ohmic-drop corrected potential vs RHE in [V]

The `plot_measurement` method always plots data (in this case potential and current) vs time.
Note that it also plots the calibrated versions (ohmic drop corrected potential and normalized current):

In [ ]:
co_strip.plot_measurement()
co_strip.plot()

Method 1: `grab()` and `np.trapz`
-----------------------------------------

Using `grab` gives you access to all of the power of `numpy` arrays. In the cells below, we use numpy functions to calculate the CO stripping charge.
Grab can select a subset of the data using a `tspan` argument. Plotting methods also can take a `tspan` argument.

So, first, we approximate the time intervals of both the stripping cycle and the base cycle:

In [ ]:
co_strip.plot_measurement(tspan=[180, 220])
co_strip.plot_measurement(tspan=[300, 340])

And then, we grab the numpy arrays:

In [ ]:
tspan_strip = [195, 215]
t_strip, I_strip = co_strip.grab("raw_current", tspan=tspan_strip)


tspan_base = [310, 330]
t_base, I_base = co_strip.grab("raw_current", tspan=tspan_base)

Printing some things to sanity check what we got:

In [ ]:
print("got these vectors for the strip:")
print(f"t/[s] = {t_strip} \nand \nI/[mA] = {I_strip}")
print()
print(f"they have these shapes: {t_strip.shape} and {I_strip.shape}")
print()
print(f"And for the base, the vectors have shapes: {t_base.shape} and {I_base.shape}")

**Checking what we've got**

Here, we manually plot the selected current vs time for the first cycle:

In [ ]:
from matplotlib import pyplot as plt

fig, ax1 = plt.subplots()
ax1.plot(t_strip, I_strip)
ax1.set_xlabel("time / [s]")
ax1.set_ylabel("current / [mA]")
ax1.set_title("stripping current")

Here, we manually plot the selected current vs time for the two cycles. Notice that of course, there is some time between them (the time corresponding to one CV cycle)

In [ ]:
fig, ax2 = plt.subplots()
ax2.plot(t_base, I_base, label="base")
ax2.plot(t_strip, I_strip, label="strip")
ax2.legend()
ax2.set_xlabel("time / [s]")
ax2.set_ylabel("current / [mA]")
ax2.set_title("strip and base current vs time")

We can use the corresponding potential to line them up on a plot. 
To get the potential corresponding to the current that we already selected, we use the method `grab_for_t`. This is when you already have the time vector and just want the values.

In [ ]:
v_strip = co_strip.grab_for_t("potential", t_strip)
v_base = co_strip.grab_for_t("potential", t_base)

fig, ax = plt.subplots()
ax.plot(v_base, I_base, color="k", label="base")
ax.plot(v_strip, I_strip, color="g", label="strip")
ax.legend()
ax.set_xlabel("potential / [V]")
ax.set_ylabel("current / [mA]")

It's not exactly aligned, so we can adjust the timespan a bit

In [ ]:
tspan_base = [311.5, 331.5]
t_base, I_base = co_strip.grab("raw_current", tspan=tspan_base)
v_base = co_strip.grab_for_t("potential", t_base)


fig, ax = plt.subplots()
ax.plot(v_base, I_base, color="k", label="base")
ax.plot(v_strip, I_strip, color="g", label="strip")
ax.legend()
ax.set_xlabel(co_strip.U_name)
ax.set_ylabel("current / [mA]")

**And do the integration!**

In [ ]:
Q_strip = np.trapz(I_strip, t_strip) * 1e-3  # converts mC --> C

Q_base = np.trapz(I_base, t_base) * 1e-3

Q_CO_ox = Q_strip - Q_base

from ixdat.constants import FARADAY_CONSTANT


#  CO + H2O --> CO2  + 2(H+ + e-)
n_CO_ox = Q_CO_ox / (FARADAY_CONSTANT * 2)

print(f"charge passed = {Q_CO_ox*1e6} uC, corresponding to {n_CO_ox*1e9} nmol of CO oxidized")


Method 2: Sweep selection and `integrate()`
----------------------------------------------------------

In [ ]:
co_strip.plot_measurement(
    J_name="cycle"
)

In [ ]:
co_strip.redefine_cycle(start_potential=0.3, redox=False)
co_strip.plot_measurement(J_name="cycle")

In [ ]:
co_strip[1].plot()

The code below selects two cycles from the CO stripping experiment. 

The code in the next bloc calculates the amount of CO according to:

$n_{CO} = \frac{1}{2 \mathcal{F}} \int_{0.6 V_{RHE}}^{1.0 V_{RHE}} ( I_{strip} - I_{base} ) \mathrm{d}t $

In [ ]:
stripping_cycle = co_strip[1]
base_cycle = co_strip[2]

ax = stripping_cycle.plot(color="green", label="strip")
base_cycle.plot(ax=ax, color="black", label="base")

ax.legend()

ax.get_figure().tight_layout()

ax.get_figure().savefig("02_two_cycles.png")

In [ ]:
vspan = [0.6, 1.0]

stripping_sweep = stripping_cycle.select_sweep(vspan=vspan)
base_sweep = base_cycle.select_sweep(vspan=vspan)

stripping_sweep  # to show what you get from this

In [ ]:
ax = stripping_sweep.plot(color="g")
base_sweep.plot(color="k", ax=ax)

In [ ]:
Q_strip = stripping_sweep.integrate("raw_current", ax="new") * 1e-3
Q_base = base_sweep.integrate("raw_current", ax="new") * 1e-3

Q_CO_ox = Q_strip - Q_base
n_CO_ox = Q_CO_ox / (FARADAY_CONSTANT * 2)

print(f"charge passed = {Q_CO_ox*1e6} uC, corresponding to {n_CO_ox*1e9} nmol of CO oxidized")

Method 3: `CyclicVoltammagramDiff`
------------------------------------------------

In [ ]:
stripping_cycle = co_strip[1]
base_cycle = co_strip[2]

ax = stripping_cycle.plot(color="g")
base_cycle.plot(ax=ax, color="k")

In [ ]:
cv_diff = stripping_cycle.diff_with(base_cycle)

cv_diff.plot()

In [ ]:
cv_diff.plot_diff()

In [ ]:
cv_diff.plot_measurement()

In [ ]:
Q_CO_ox = cv_diff.integrate("raw_current", vspan=[0.6, 1.0]) * 1e-3  # 1e-3 converts mC --> C
n_CO_ox = Q_CO_ox / (FARADAY_CONSTANT * 2)

print(f"charge passed = {Q_CO_ox*1e6} uC, corresponding to {n_CO_ox*1e9} nmol of CO oxidized")

Your turn!
========

In [ ]:
# oxide_reduction = CyclicVoltammagram.read(data_directory / "oxide_reduction.csv", reader="ixdat")
if True:  # Set this to False for offline work (requires you have downloaded the data file.)
    oxide_reduction = CyclicVoltammogram.read_url(
        "https://raw.githubusercontent.com/ixdat/tutorials/43e85d07254e67e950f9c7081ffe1fa7f053cc08/L3_data_structure/exports/oxide_reduction.csv",
        reader="ixdat"
    )
else:
    oxide_reduction = CyclicVoltammogram.read(
        "./data/oxide_reduction.csv", reader="ixdat"
    )
    
oxide_reduction.calibrate(A_el=0.196, R_Ohm=100)

oxide_reduction.tstamp += oxide_reduction.t[0]
oxide_reduction.plot_measurement()
oxide_reduction.plot(tspan=[300, 800])